In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from make_prediction import DataSplits

In [ ]:
particles = ['kaon', 'pion', 'muon', 'proton']

In [ ]:
i_plot = 0
plt.figure(figsize=(90, 70))
for p in particles:
    df = pd.read_csv('validation_output/{}.csv'.format(p))
    
    pred_cols = [col for col in df.columns]
    
    for col in pred_cols:
        i_plot += 1
        plt.subplot(len(particles), len(pred_cols), i_plot)
        plt.hist(df[col], bins=80)
        plt.title(p)
        plt.xlabel(col);

In [ ]:
import utils_rich_mrartemev as utils_rich

data_full = {
    particle : utils_rich.load_and_merge_and_cut(utils_rich.datasets[particle])
    for particle in particles
}

In [ ]:
plt.figure(figsize=(9, 6))
for particle in particles:
    x = data_full[particle]['nTracks_Brunel']
    w = data_full[particle][utils_rich.weight_col]
    plt.hist(x, weights=w, bins=np.linspace(0, 600, 201), density=True, label=particle, histtype='step')

plt.xlabel('nTracks')
plt.legend()
plt.show();

In [ ]:
data_ref  = pd.read_pickle("predictions.pkl")
data_test = pd.read_pickle("predictions_with_tfScaler.pkl")

In [ ]:
i_plot = 0
plt.figure(figsize=(90, 70))
for p in particles:
    assert set(data_ref[p].test.columns) == set(data_test[p].test.columns)
    cols = [col for col in data_ref[p].test.columns if 'predicted' not in col]
    assert (data_ref[p].test[cols] == data_test[p].test[cols]).all().all()
    
    pred_cols = [col for col in data_ref[p].test.columns if 'predicted' in col]
    
    for col in pred_cols:
        i_plot += 1
        plt.subplot(len(particles), len(pred_cols), i_plot)
        ref  = data_ref [p].test[col]
        test = data_test[p].test[col]
        
        bins = np.linspace(
            min(ref.min(), test.min()),
            max(ref.max(), test.max()),
            101
        )
        plt.hist(ref , bins=bins, label='ref')
        plt.hist(test, bins=bins, label='test', histtype='step')
        plt.legend();